In [1]:
%matplotlib widget

In [2]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import scipy.signal as sctrl
import pydae.ssa as ssa
from sympy.physics.quantum import TensorProduct

# Control in real domain



### Plant model

In [3]:
Δt = 50e-6
R_t =  0.039269908169872414
L_t =  0.00125
C_m =  4e-06
G_d =  1.0
R_s =  0.039269908169872414
L_s =  0.00125
omega = 2*np.pi*50

R = R_t + R_s
L = L_t + L_s
A =  np.array([
    [  -R/L, omega],
    [-omega,  -R/L],
    ])

B = np.array([
    [ 1/L,0],
    [ 0,1/L],
    ])

B_g = np.array([
    [ -1/L,0],
    [ 0,-1/L],
    ])

C_c = np.array([
    [ 1, 0],
    [ 0, 1],
    ])

D_c = np.array([
    [ 0, 0],
    [ 0, 0],
    ])

C_o = np.array([
    [ 1],
    ])

D_o = np.array([
    [ 0, 0],
    [ 0, 0],
    ])

# plant discretization
A_d,B_d,C_d,D_d,Dt = sctrl.cont2discrete((A,B,C_c,D_c),Δt,method='zoh')
A_,B_gd,C_,D_,Dt   = sctrl.cont2discrete((A,B_g,C_c,D_c),Δt,method='zoh')
A_,B_,C_o,D_o,Dt   = sctrl.cont2discrete((A,B,C_o,D_o),Δt,method='zoh')


## Controller

In [33]:
def acker(A,B,poles):
    '''
    This function is a copy from the original in: https://github.com/python-control/python-control
    but it allows to work with complex A and B matrices. It is experimental and the original should be
    considered
    
    
    ----------
    A : numpy array_like (complex can be used)
        Dynamics amatrix of the system
    B : numpy array_like (complex can be used)
        Input matrix of the system
    poles : numpy array_like
        Desired eigenvalue locations.

    Returns
    -------
    K : numpy array_like
        Gain such that A - B K has eigenvalues given in p.

    '''
    
    
    N_x = np.shape(A)[0]
    
    ctrb = np.hstack([B] + [np.dot(np.linalg.matrix_power(A, i), B)
                                             for i in range(1, N_x)])

    # Compute the desired characteristic polynomial
    p = np.real(np.poly(poles))

    n = np.size(p)
    pmat = p[n-1] * np.linalg.matrix_power(A, 0)
    for i in np.arange(1,n):
        pmat = pmat + np.dot(p[n-i-1], np.linalg.matrix_power(A, i))
    K = np.linalg.solve(ctrb, pmat)

    K = K[-1][:]                # Extract the last row            # Extract the last row
    
    return K

In [35]:
# Controller ##################################################################################
N_x_c,N_u_d = B_d.shape
N_z_c,N_x_c = C_c.shape


O_ux = np.zeros((N_u_d,N_x_c))
O_xu = np.zeros((N_x_c,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)


# discretized plant:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δz_c = C_c*Δx_d + D_c*Δu_d

# dinamic extension:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δx_i = Δx_i + Δt*(Δz_c-Δz_c_ref) = Δx_i + Δt*C_c*Δx_d - Dt*Δz_c_ref
# Δz_c = z_c - z_c_0
# Δz_c_ref = z_c_ref - z_c_0
# (Δz_c-Δz_c_ref) = z_c - z_c_ref


A_e = np.block([
                [    A_d,    B_d, O_xu],    # Δx_d
                [   O_ux,   O_uu, O_uu],    # Δx_r
                [ Δt*C_d,   O_uu, I_uu],    # Δx_i    
               ])

B_e = np.block([
                [   O_xu],
                [   I_uu],
                [   O_uu],    
               ])



# weighting matrices
Q_c = np.eye(A_e.shape[0])
Q_c[-1,-1] = 1e7
Q_c[-2,-2] = 1e7

R_c = np.eye(B_e.shape[1])*10

K_c,S_c,E_c = ssa.dlqr(A_e,B_e,Q_c,R_c)

E_cont = np.log(E_c)/Δt


gamma = np.exp(-1000*50.0e-6)  
poles = np.array([gamma,gamma]*3);

K_c = acker(A_e,B_e,poles)



LinAlgError: Last 2 dimensions of the array must be square

In [32]:
B_e

array([[0., 0.],
       [0., 0.],
       [1., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.]])

In [24]:
N_x = 2
N_r = 2
N_i = 2

Dx_d = sym.Matrix([sym.Symbol(f'Dx_d[{it}]', real=True) for it in range(N_x)])
Dx_r = sym.Matrix([sym.Symbol(f'Dx_r[{it}]', real=True) for it in range(N_r)])
Dx_i = sym.Matrix([sym.Symbol(f'Dx_i[{it}]', real=True) for it in range(N_i)])

Dx_e = sym.Matrix([[Dx_d],[Dx_r],[Dx_i]])
                   
Du_r = -K_c @ Dx_e

Du_r = sym.N(sym.simplify(u_r))

for it in range(2):
    print(f'Du_r[{it}] = {str(sym.N(Du_r[it],8))};')
 
print('\nWarning: Control output is v_t_dq!!')

Du_r[0] = -2.1004933*Dx_d[0] - 0.015941738*Dx_d[1] - 876.73711*Dx_i[0] + 322.0766*Dx_i[1] - 0.041168797*Dx_r[0] - 0.00031799792*Dx_r[1];
Du_r[1] = 0.015941738*Dx_d[0] - 2.1004933*Dx_d[1] - 322.0766*Dx_i[0] - 876.73711*Dx_i[1] + 0.00031799792*Dx_r[0] - 0.041168797*Dx_r[1];



## Controller in complex domain without delay

### Plant model

In [36]:
Δt = 50e-6
R_t =  0.039269908169872414
L_t =  0.00125
C_m =  4e-06
G_d =  1.0
R_s =  0.039269908169872414
L_s =  0.00125
omega = 2*np.pi*50

R = R_t + R_s
L = L_t + L_s
A =  np.array([
    [-R/L-1j*omega],
    ])

B = np.array([
    [ 1/L],
    ])

B_g = np.array([
    [ -1/L],
    ])

C_c = np.array([
    [ 1],
    ])

D_c = np.array([
    [ 0],
    ])

C_o = np.array([
    [ 1],
    ])

D_o = np.array([
    [ 0],
    ])

# plant discretization
A_d,B_d,C_d,D_d,Dt = sctrl.cont2discrete((A,B,C_c,D_c),Δt,method='zoh')
A_,B_gd,C_,D_,Dt   = sctrl.cont2discrete((A,B_g,C_c,D_c),Δt,method='zoh')
A_,B_,C_o,D_o,Dt   = sctrl.cont2discrete((A,B,C_o,D_o),Δt,method='zoh')


In [37]:
# Controller ##################################################################################
N_x_c,N_u_d = B_d.shape
N_z_c,N_x_c = C_c.shape


O_ux = np.zeros((N_u_d,N_x_c))
O_xu = np.zeros((N_x_c,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)


# discretized plant:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δz_c = C_c*Δx_d + D_c*Δu_d

# dinamic extension:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δx_i = Δx_i + Δt*(Δz_c-Δz_c_ref) = Δx_i + Δt*C_c*Δx_d - Dt*Δz_c_ref
# Δz_c = z_c - z_c_0
# Δz_c_ref = z_c_ref - z_c_0
# (Δz_c-Δz_c_ref) = z_c - z_c_ref



A_e = np.block([
                [    A_d,   O_xu],    # Δx_d
                [ Δt*C_d,   I_uu],    # Δx_i    
               ])

B_e = np.block([
                [   B_d],
                [   O_uu],    
               ])

# weighting matrices
Q_c = np.eye(A_e.shape[0])
Q_c[-1,-1] = 1e7


R_c = np.eye(B_e.shape[1])*10

K_c,S_c,E_c = ssa.dlqr(A_e,B_e,Q_c,R_c)

E_cont = np.log(E_c)/Δt


gamma = np.exp(-1000*50.0e-6)  
poles = np.array([gamma,gamma]);

K_c = ssa.acker(A_e,B_e,poles).reshape(1,2)

In [38]:
Dx_e

Matrix([
[Dx_d[0]],
[Dx_d[1]],
[Dx_r[0]],
[Dx_r[1]],
[Dx_i[0]],
[Dx_i[1]]])

In [39]:
N_x_cplx = 1
N_i_cplx = 1

Dx_d = sym.Matrix([sym.Symbol(f'Dx_d[{it}]', real=True) + 1j*sym.Symbol(f'Dx_d[{it+1}]', real=True)  for it in range(0,N_x_cplx,2)])
Dx_i = sym.Matrix([sym.Symbol(f'Dx_i[{it}]', real=True) + 1j*sym.Symbol(f'Dx_i[{it+1}]', real=True)  for it in range(0,N_i_cplx,2)])

Dx_e = sym.Matrix([[Dx_d],[Dx_i]])
  
   


                   
u_r = -K_c @ Dx_e

Du_r_1 = str(sym.N(sym.simplify(sym.re(u_r[0])),8))
Du_r_2 = str(sym.N(sym.simplify(sym.im(u_r[0])),8))

print(f'Du_r[0] = {Du_r_1};')
print(f'Du_r[1] = {Du_r_2};')
print('\nWarning: Control output is v_t_dq!!')

Du_r[0] = -4.8022489*Dx_d[0] - 0.74707379*Dx_d[1] - 2380.3887*Dx_i[0] + 18.691019*Dx_i[1];
Du_r[1] = 0.74707379*Dx_d[0] - 4.8022489*Dx_d[1] - 18.691019*Dx_i[0] - 2380.3887*Dx_i[1];



In [70]:
K_c

array([   4.80224889 -0.74707379j, 2380.3887398 +18.69101907j])

## Controller

In [16]:
# Controller ##################################################################################
N_x_c,N_u_d = B_d.shape
N_z_c,N_x_c = C_c.shape


O_ux = np.zeros((N_u_d,N_x_c))
O_xu = np.zeros((N_x_c,N_u_d))
O_uu = np.zeros((N_u_d,N_u_d))
I_uu = np.eye(N_u_d)


# discretized plant:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δz_c = C_c*Δx_d + D_c*Δu_d

# dinamic extension:
# Δx_d = A_d*Δx_d + B_d*Δu_d
# Δx_i = Δx_i + Δt*(Δz_c-Δz_c_ref) = Δx_i + Δt*C_c*Δx_d - Dt*Δz_c_ref
# Δz_c = z_c - z_c_0
# Δz_c_ref = z_c_ref - z_c_0
# (Δz_c-Δz_c_ref) = z_c - z_c_ref


A_e = np.block([
                [    A_d,    B_d, O_xu],    # Δx_d
                [   O_ux,   O_uu, O_uu],    # Δx_r
                [ Δt*C_d,   O_uu, I_uu],    # Δx_i    
               ])

B_e = np.block([
                [   O_xu],
                [   I_uu],
                [   O_uu],    
               ])

A_e = np.block([
                [    A_d,   O_xu],    # Δx_d
                [ Δt*C_d,   I_uu],    # Δx_i    
               ])

B_e = np.block([
                [   B_d],
                [   O_uu],    
               ])

# weighting matrices
Q_c = np.eye(A_e.shape[0])
Q_c[-1,-1] = 1e7


R_c = np.eye(B_e.shape[1])*10

K_c,S_c,E_c = ssa.dlqr(A_e,B_e,Q_c,R_c)

E_cont = np.log(E_c)/Δt



In [24]:
K

array([ 5.00382521e+01+3.92904700e-01j, -9.03149268e+05-7.09160647e+03j])

In [199]:
A_e

array([[9.98307263e-01-0.01568266j, 0.00000000e+00+0.j        ],
       [5.00000000e-05+0.j        , 1.00000000e+00+0.j        ]])

In [164]:
# Control without observer Du_r = -K_c*Dx_e
Dx_d_1,Dx_d_2 = sym.symbols('Dx_d_1,Dx_d_2', real=True)
Dx_r_1,Dx_r_2 = sym.symbols('Dx_r_1,Dx_r_2', real=True)
Dx_i_1,Dx_i_2 = sym.symbols('Dx_i_1,Dx_i_2', real=True)

Dx_d_12 = Dx_d_1 + 1j*Dx_d_2
Dx_r_12 = Dx_r_1 + 1j*Dx_r_2
Dx_i_12 = Dx_i_1 + 1j*Dx_i_2


   
   
K_c = np.array([4.8037 - 0.7471j, 2.3818e+03 + 1.8702e+01j ]).reshape(1,2)
   
x_e = sym.Matrix([Dx_d_12,Dx_r_12,Dx_i_12])
x_e = sym.Matrix([Dx_d_12,Dx_i_12])
u_r = -K_c @ x_e

Du_r_1 = str(sym.N(sym.simplify(sym.re(u_r[0])),8))
Du_r_2 = str(sym.N(sym.simplify(sym.im(u_r[0])),8))

print(f'Du_r_1 = {Du_r_1};')
print(f'Du_r_2 = {Du_r_2};')
print('\nWarning: Control output is v_t_dq!!')

Du_r_1 = -4.8037*Dx_d_1 - 0.7471*Dx_d_2 - 2381.8*Dx_i_1 + 18.702*Dx_i_2;
Du_r_2 = 0.7471*Dx_d_1 - 4.8037*Dx_d_2 - 18.702*Dx_i_1 - 2381.8*Dx_i_2;



In [159]:
omega*L

0.7853981633974483

In [76]:
E_cont

array([-7.13979286e+05-58453.58811349j, -4.30854602e+02  +604.05182493j,
       -4.24472904e+02  -289.89255957j])

## Observer

In [10]:
N_z_o = C_o.shape[0]

Q_o = np.eye(A_d.shape[0])

R_o = np.diag([1]*N_z_o)
K_o_T,S_o,E_o = ssa.dlqr(A_d.T,C_o.T,Q_o,R_o)
K_o = K_o_T.T


print('damp_ctrl',-E_c.real/np.abs(E_c))
print('damp_obs',-E_o.real/np.abs(E_o))

damp_ctrl [-0.99954394 -0.99954394 -0.99989495 -0.99989495 -1.          1.        ]
damp_obs [-0.99987663]


## Simulink

In [78]:
# Control without observer Du_r = -K_c*Dx_e
Dx_d_1 = sym.symbols('Dx_d_1')
Dx_r_1 = sym.symbols('Dx_r_1')
Dx_i_1 = sym.symbols('Dx_i_1')

x_e = sym.Matrix([Dx_d_1,Dx_r_1,Dx_i_1 ])
u_r = -K_c * x_e

u_r_d = str(sym.N(u_r[0],8))

print(f'Du_r_1 = {u_r_d};')

print('\nWarning: Control output is v_t_dq!!')

Du_r_1 = Dx_d_1*(-2.1004933 - 0.015941738*I) + Dx_i_1*(-876.73711 + 322.0766*I) + Dx_r_1*(-0.041168797 - 0.00031799792*I);



In [32]:
from sympy.utilities.codegen import codegen
from sympy.abc import f, g

from sympy import Eq

[(c_name, c_code), (h_name, c_header)] = codegen(

     [("myfcn", u_r_d)],

     "C99", header=False, empty=False)

print(c_code)



#include "myfcn.h"
#include <math.h>
double myfcn(double Dx_d_1, double Dx_i_1, double Dx_r_1) {
   double myfcn_result;
   myfcn_result = Dx_d_1*(-0.0032323944000000001 + 2.3373778999999999e-5*I) + Dx_i_1*(-0.10354921 + 0.99459178000000004*I) + Dx_r_1*(-6.4596544999999994e-5 - 2.0057920000000001e-8*I);
   return myfcn_result;
}



In [29]:
it_ini = 4
Δx_o = sym.Matrix([sym.Symbol(f'xD[{it+it_ini}]') for it in range(6)])
Δz_o = sym.Matrix([sym.Symbol(item) - sym.Symbol(item+'_0') for item in ['v_md', 'v_mq', 'i_sd', 'i_sq']])
Δx_r = sym.Matrix([sym.Symbol(f'Dx_r_{it+1}') for it in range(2)])
Δu_pert = sym.Matrix([sym.Symbol(f'Du_pert_{it+1}') for it in range(2)])

Δx_o_kp1 = A_d @ Δx_o + B_b@(Δx_r) +  K_o @ (Δz_o - C_o @ Δx_o) # + B_pert@Δu_pert  +  K_o @ (Δz_o - C_o @ Δx_o - D_o @ (Δx_r))

for it in range(6):
    print(f'xD[{it+it_ini}] = {Δx_o_kp1[it]};')

xD[4] = 0.0392952872921423*Dx_r_1 + 0.000617299701089028*Dx_r_2 + 0.167670572903802*i_sd - 0.167670572903802*i_sd_0 + 0.427599429013122*v_md - 0.427599429013122*v_md_0 + 0.962811636697318*xD[4] - 0.430586049216921*xD[6] - 0.132051772873075*xD[8];
xD[5] = -0.000617299701089028*Dx_r_1 + 0.0392952872921423*Dx_r_2 + 0.167670572903802*i_sq - 0.167670572903802*i_sq_0 + 0.427599429013122*v_mq - 0.427599429013122*v_mq_0 + 0.962811636697318*xD[5] - 0.430586049216921*xD[7] - 0.132051772873075*xD[9];
xD[6] = 0.0356406531431186*Dx_r_1 + 0.000559888120127431*Dx_r_2 - 0.425448653071038*i_sd + 0.425448653071038*i_sd_0 + 0.443158057967852*v_md - 0.443158057967852*v_md_0 + 0.933318813687106*xD[4] - 0.449166750687225*xD[6] - 0.507870160616068*xD[8];
xD[7] = -0.000559888120127431*Dx_r_1 + 0.0356406531431186*Dx_r_2 - 0.425448653071038*i_sq + 0.425448653071038*i_sq_0 + 0.443158057967853*v_mq - 0.443158057967853*v_mq_0 + 0.933318813687106*xD[5] - 0.449166750687225*xD[7] - 0.507870160616068*xD[9];
xD[8] = 0.

In [30]:
#eta_dq[0] = Du_r_1*2/v_dc[0] + (v_sd)*2/v_dc[0] ; 
#eta_dq[1] = Du_r_2*2/v_dc[0] + (v_sq)*2/v_dc[0] ;
#Du_r_1 = eta_dq[0]*v_dc[0]/2
#Du_r_2 = eta_dq[1]*v_dc[0]/2

Du_r_1,Du_r_2 = sym.symbols('Du_r_1,Du_r_2')
Du_r = sym.Matrix([Du_r_1,Du_r_2 ])

Dx_r = W@Du_r

Dx_r_1 = str(sym.N(Dx_r[0],8))
Dx_r_1 = str(sym.N(Dx_r[1],8))

print(f'Dx_r_1 = {Dx_r_1};')
print(f'Dx_r_2 = {Dx_r_1};')

Dx_r_1 = -0.015707317*Du_r_1 + 0.99987663*Du_r_2;
Dx_r_2 = -0.015707317*Du_r_1 + 0.99987663*Du_r_2;


In [32]:
# Control with observer Du_r = -K_c*Dx_o
x_d_1,x_d_2,x_d_3,x_d_4,x_d_5,x_d_6 = sym.symbols('xD[4],xD[5],xD[6],xD[7],xD[8],xD[9]')
x_r_1,x_r_2 = sym.symbols('Dx_r_1,Dx_r_2')
x_i_1,x_i_2 = sym.symbols('Dx_i_1,Dx_i_2')

x_e = sym.Matrix([x_d_1,x_d_2,x_d_3,x_d_4,x_d_5,x_d_6,x_r_1,x_r_2,x_i_1,x_i_2])
u_r = -K_c * x_e

u_r_d = str(sym.N(u_r[0],8))
u_r_q = str(sym.N(u_r[1],8))

print(f'Du_r_1 = {u_r_d};')
print(f'Du_r_2 = {u_r_q};')

print('\nWarning: Control output is eta_dq!!')

Du_r_1 = -2715.7977*Dx_i_1 + 1040.9568*Dx_i_2 - 0.071871552*Dx_r_1 + 0.00052148736*Dx_r_2 - 1.8262175*xD[4] + 0.019754652*xD[5] - 0.0048070784*xD[6] + 0.0010494948*xD[7] - 3.4543535*xD[8] + 0.39761572*xD[9];
Du_r_2 = -1040.9568*Dx_i_1 - 2715.7977*Dx_i_2 - 0.00052148736*Dx_r_1 - 0.071871552*Dx_r_2 - 0.019754652*xD[4] - 1.8262175*xD[5] - 0.0010494948*xD[6] - 0.0048070784*xD[7] - 0.39761572*xD[8] - 3.4543535*xD[9];



## Symbolic obtention of the plant model

In [ ]:
R_t,L_t,C_m,G_d,R_s,L_s = sym.symbols('R_t,L_t,C_m,G_d,R_s,L_s', real=True)
i_tD,i_tQ,v_mD,v_mQ,i_sD,i_sQ = sym.symbols('i_tD,i_tQ,v_mD,v_mQ,i_sD,i_sQ', real=True)
v_tD,v_tQ = sym.symbols('v_tD,v_tQ', real=True)
v_sD,v_sQ = sym.symbols('v_sD,v_sQ', real=True)


In [44]:
R_t,L_t,C_m,G_d,R_s,L_s = sym.symbols('R_t,L_t,C_m,G_d,R_s,L_s', real=True)
i_tD,i_tQ,v_mD,v_mQ,i_sD,i_sQ = sym.symbols('i_tD,i_tQ,v_mD,v_mQ,i_sD,i_sQ', real=True)
v_tD,v_tQ = sym.symbols('v_tD,v_tQ', real=True)
v_sD,v_sQ = sym.symbols('v_sD,v_sQ', real=True)

A = sym.Matrix([
    [-R_t/L_t,       0,   -1/L_t,        0,        0,        0],
    [       0,-R_t/L_t,        0,   -1/L_t,        0,        0],
    [   1/C_m,       0, -G_d/C_m,        0,   -1/C_m,        0],
    [       0,   1/C_m,        0, -G_d/C_m,        0,   -1/C_m],
    [       0,       0,    1/L_s,        0, -R_s/L_s,        0],
    [       0,       0,        0,    1/L_s,        0, -R_s/L_s],
    ])

B = sym.Matrix([
    [ 1/L_t,        0],
    [       0,  1/L_t],
    [       0,      0],
    [       0,      0],
    [       0,      0],
    [       0,      0],
    ])

B_g = sym.Matrix([
    [       0,      0],
    [       0,      0],
    [       0,      0],
    [       0,      0],
    [-1/L_s,        0],
    [       0, -1/L_s],
    ])

x = sym.Matrix([i_tD,i_tQ,v_mD,v_mQ,i_sD,i_sQ])
u = sym.Matrix([v_tD,v_tQ])
u_g = sym.Matrix([v_sD,v_sQ])

dx_new = A@x + B@u + B_g@u_g

di_tD = 1/L_t*(v_tD - R_t*i_tD - v_mD)  
di_tQ = 1/L_t*(v_tQ - R_t*i_tQ - v_mQ) 
dv_mD = 1/C_m*(i_tD - G_d*v_mD - i_sD) 
dv_mQ = 1/C_m*(i_tQ - G_d*v_mQ - i_sQ) 
di_sD = 1/L_s*(v_mD - R_s*i_sD - v_sD)  
di_sQ = 1/L_s*(v_mQ - R_s*i_sQ - v_sQ)
dx = sym.Matrix([di_tD,di_tQ,dv_mD,dv_mQ,di_sD,di_sQ])

dx = sym.Matrix([di_tD,di_tQ,dv_mD,dv_mQ,di_sD,di_sQ])
sym.simplify(dx_new - dx)  # just to check the model is ok

Matrix([
[0],
[0],
[0],
[0],
[0],
[0]])

In [40]:
import os


In [41]:
os.name

'nt'

FloatProgress(value=0.0, max=1.0)

In [32]:
time.sleep?

Docstring:
sleep(seconds)

Delay execution for a given number of seconds.  The argument may be
a floating point number for subsecond precision.
Type:      builtin_function_or_method
